In [1]:
# coding: utf-8
import argparse
import glob
import librosa
from tensorflow.keras.layers import BatchNormalization, Multiply, Add
from tensorflow.keras.layers import Conv1D
from tensorflow. keras.utils import multi_gpu_model
import tensorflow.keras.callbacks
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import add, concatenate
from tensorflow.keras.layers import Reshape, Lambda
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow.keras.backend as K
import datetime
import re
import itertools
import platform
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from sklearn.metrics import accuracy_score
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import os
import codecs
import numpy as np
from os.path import join
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tensorflow.compat.v1.keras.backend.set_session(sess)
tensorflow.compat.v1.keras.backend.clear_session()  # 清理session

np.random.seed(2018)

Instructions for updating:
non-resource variables are not supported in the long term



In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 　选择使用的GPU
path_base = './dataset/data_thchs30'
path_data = join(path_base, 'data')
text_files = glob.glob(join(path_data, '*.wav.trn'))
text_files.sort()
text_files = text_files[:3]
text_files[0]

'./dataset/data_thchs30\\data\\A11_0.wav.trn'

In [5]:
lines = []
for text_file in tqdm(text_files):
    with codecs.open(text_file, encoding='utf-8') as f_read:
        line = f_read.readline()
        lines.append(line.strip().replace(" ", ""))
lines

100%|██████████| 3/3 [00:00<00:00, 96.80it/s]


['绿是阳春烟景大块文章的底色四月的林峦更是绿得鲜活秀媚诗意盎然',
 '他仅凭腰部的力量在泳道上下翻腾蛹动蛇行状如海豚一直以一头的优势领先',
 '炮眼打好了炸药怎么装岳正才咬了咬牙倏地脱去衣服光膀子冲进了水窜洞']

In [8]:
text_lines = lines 
tok_path = join(path_base, 'tokenizer.pickle')
tok = Tokenizer(char_level=True)
tok.fit_on_texts(text_lines)

seq_lines = tok.texts_to_sequences(text_lines[:])
print(seq_lines)

[[3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 1, 15, 16, 17, 18, 1, 19, 20, 21, 4, 3, 22, 23, 24, 25, 26, 27, 28, 29, 30], [31, 32, 33, 34, 35, 1, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5, 53, 54, 5, 55, 1, 56, 57, 58, 59], [60, 61, 62, 63, 2, 64, 65, 66, 67, 68, 69, 70, 71, 6, 2, 6, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 2, 84, 85, 86]]


In [10]:
len_lines = pd.Series(map(lambda x: len(x), seq_lines))
len_lines

0    30
1    33
2    32
dtype: int64

In [12]:
maxlen=48
lines = seq_lines[:]
pad_lines = pad_sequences(lines, maxlen=maxlen, padding="post", truncating="pre")
pad_lines

array([[ 3,  4,  7,  8,  9, 10, 11, 12, 13, 14,  1, 15, 16, 17, 18,  1,
        19, 20, 21,  4,  3, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [31, 32, 33, 34, 35,  1, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52,  5, 53, 54,  5, 55,  1, 56, 57, 58,
        59,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [60, 61, 62, 63,  2, 64, 65, 66, 67, 68, 69, 70, 71,  6,  2,  6,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,  2, 84, 85, 86,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])